# 1. 사례: 전략 패턴의 리팩토링

## 1.1 고전적인 전략

In [2]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

In [3]:
class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [4]:
class Promotion(ABC):  # the Strategy: an Abstract Base Class

    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [8]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [9]:
cart = [LineItem('banana', 4, .5), 
       LineItem('apple', 10, 1.5),
       LineItem('watermelon', 5, 5.0)]

In [11]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [12]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [15]:
banana_cart = [LineItem('banana', 30, .5),  # <5>
                LineItem('apple', 10, 1.5)]

In [16]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [21]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [27]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [28]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

## 1.2 함수지향 전략

In [37]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

In [38]:
class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [39]:
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [40]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [41]:
cart = [LineItem('banana', 4, .5), 
       LineItem('apple', 10, 1.5),
       LineItem('watermelon', 5, 5.0)]

In [42]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

## 1.3 최선의 전략 선택하기: 단순한 접근법

In [50]:

from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [ ]:
# 함수 리스트를 반복해서 최대 할인액을 찾아내는 best_promo() 함수
promos = [fidelity_promo, bulk_item_promo, large_order_promo]  # <1>

def best_promo(order):  # <2>
    """Select best discount available
    """
    return max(promo(order) for promo in promos)  # <3>


## 1.4 모듈에서 전략 찾기

In [59]:
# 6-7 모듈 전역 네임스페이스를 내부 조사해서 만든 promos 리스트
promos = [globals()[name] for name in globals()  # <1>
            if name.endswith('_promo')  # <2>
            and name != 'best_promo']   # <3>

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)  # <4>

In [60]:
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [61]:
[name for name in globals() if name.endswith('_promo')]

['fidelity_promo', 'bulk_item_promo', 'large_order_promo', 'best_promo']

In [62]:
globals()['large_order_promo']

<function __main__.large_order_promo(order)>

In [ ]:
!pip pro

In [63]:
# 6-8 새로운 promotions 모듈을 내부 조사해서 만든 promos 리스트

In [74]:
from promotions import *

ModuleNotFoundError: No module named 'promotions'

In [73]:
import inspect
import promotions

promos = [func for name, func in
                inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

ModuleNotFoundError: No module named 'promotions'

# 2. 명령

In [76]:
class MacroCommand:
    """명령 리스트를 실행하는 명령"""
    
    def __init__(self, commands):
        self.commands = list(commands)
        
    def __call__(self):
        for command in self.commands:
            command()